In [1]:
using Random
using HDF5
using Distributed
using Base.Iterators: flatten
using ProgressMeter
using JSON
using DataFrames
using CSV
using Reexport
using StatsBase
using PyFormattedStrings
include("simple_tree.jl")
include("zss.jl")

tree_edit_distance (generic function with 1 method)

In [2]:
Random.seed!(42)

function rev_bfs(tree, root_index=0)
    rbfs = Int[]
    queue = [root_index]
    while !isempty(queue)
        current_index = popfirst!(queue)
        unshift!(rbfs, current_index)
        append!(queue, tree[current_index]["children"])
    end
    return rbfs
end

rev_bfs (generic function with 2 methods)

In [3]:
function get_bin(lst, K)
    for i in 1:length(lst)
        if K < lst[i]
            return lst[i-1]
        end
    end
    if K == lst[end]
        return lst[end]
    else
        return nothing
    end
end

get_bin (generic function with 1 method)

In [4]:
function parse_graphviz_tree_zss(gv_tree::String; root_index=0, constant_bins=nothing, args...)
    tree = Dict{Int, Dict}()
    nodes = matchall(r"^(.*?)(?=\s+fillcolor)", gv_tree, RegexOpts("m"))
    for i in 1:length(nodes)
        split_node = split(nodes[i], r" \[label=")
        node_index = parse(Int, split_node[1])
        node_value = replace(split_node[2], r"," => "")
        tree[node_index] = Dict("depth" => 1, "value" => node_value, "children" => Int[])
    end

    children = matchall(r"\d+ -> \d+", gv_tree)
    for i in 1:length(children)
        split_child = split(children[i], r" -> ")
        parent = parse(Int, split_child[1])
        child = parse(Int, split_child[2])
        push!(tree[parent]["children"], child)
        sort!(tree[parent]["children"])
    end

    starting_index = minimum(keys(tree))
    stack = [starting_index]
    tree_height = 0
    while !isempty(stack)
        current_node = popfirst!(stack)
        current_depth = tree[current_node]["depth"]
        tree_height = max(tree_height, current_depth)
        for child in tree[current_node]["children"]
            tree[child]["depth"] = current_depth + 1
            push!(stack, child)
        end
    end

    indexes = rev_bfs(tree)
    zss_list = fill(nothing, length(indexes))

    for i in indexes
        try
            tree[i]["value"] = parse(Float64, tree[i]["value"])
        catch e
            if isa(e, ArgumentError)
                # do nothing
            else
                rethrow(e)
            end
        end

        if constant_bins !== nothing
            tree[i]["value"] = "const_$(get_bin(constant_bins, tree[i]["value"]))"
        end

        if !isempty(tree[i]["children"])
            zss_list[i] = Node(tree[i]["value"], [zss_list[c] for c in tree[i]["children"]])
        else
            zss_list[i] = Node(tree[i]["value"])
        end
    end

    return zss_list[1]
end

parse_graphviz_tree_zss (generic function with 1 method)

In [5]:
function generate_pairs(n::Int)
    return [(i, j) for i in 1:n for j in i+1:n]
end

generate_pairs (generic function with 1 method)

In [6]:
function write_results_to_hdf5(hdf5_file, results)
    for ((i, j), distance) in results
        row_start_index = i * (i + 1) ÷ 2
        hdf5_file["distance_matrix"][row_start_index + j + 1] = distance
    end
end

write_results_to_hdf5 (generic function with 1 method)

In [7]:
folder_path = "../Data/Graphviz Tree/"
files_json = filter(f -> endswith(f, ".json"), readdir(folder_path))
files = (f -> joinpath(folder_path, f)).(files_json)

4-element Vector{String}:
 "../Data/Graphviz Tree/graphviz_tree_no_tag_FPI_ml-prove.json"
 "../Data/Graphviz Tree/graphviz_tree_no_tag_FPI_mushrooms.json"
 "../Data/Graphviz Tree/graphviz_tree_with_tag_FPI_ml-prove.json"
 "../Data/Graphviz Tree/graphviz_tree_with_tag_FPI_mushrooms.json"

In [8]:
file = files[1]

"../Data/Graphviz Tree/graphviz_tree_no_tag_FPI_ml-prove.json"

In [9]:
graphviz_tree_list = open(file) do f
    JSON.parse(f)
end

69960-element Vector{Any}:
 "digraph Program {\n\tnode [style=" ⋯ 320 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 320 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 320 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 324 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 324 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 324 bytes ⋯ "> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 472 bytes ⋯ "> 6\n\t3 -> 7\n\t3 -> 8\n\t3 -> 9\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 472 bytes ⋯ "> 6\n\t3 -> 7\n\t3 -> 8\n\t3 -> 9\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 472 bytes ⋯ "> 6\n\t3 -> 7\n\t3 -> 8\n\t3 -> 9\n}\n"
 "digraph Program {\n\tnode [style=" ⋯ 515 bytes ⋯ " 7\n\t3 -> 8\n\t3 -> 9\n\t3 -> 10\n}\n"
 ⋮
 "digraph Program {\n\tnode [style=" ⋯ 658 bytes ⋯

In [10]:
base_name = splitext(basename(file))[1]
pattern = r"_FPI_([^_]+)"
dataset_match = match(pattern, base_name)
base_match = dataset_match.captures[1]

output_dir = joinpath("..", "Data", "Distancias", "Edit Tree Teste Julia")
mkpath(output_dir)

"..\\Data\\Distancias\\Edit Tree Teste Julia"

In [11]:
indices_folder = "../Data/Small Sample Index"
indices_files_csv = filter(f -> endswith(f, ".csv") && occursin(base_match, splitext(basename(f))[1]), readdir(indices_folder))
indices_files = (f -> joinpath(indices_folder, f)).(indices_files_csv)

5-element Vector{String}:
 "../Data/Small Sample Index\\sample_index_1_FPI_ml-prove.csv"
 "../Data/Small Sample Index\\sample_index_2_FPI_ml-prove.csv"
 "../Data/Small Sample Index\\sample_index_3_FPI_ml-prove.csv"
 "../Data/Small Sample Index\\sample_index_4_FPI_ml-prove.csv"
 "../Data/Small Sample Index\\sample_index_5_FPI_ml-prove.csv"

In [12]:
indice_file = indices_files[1]


"../Data/Small Sample Index\\sample_index_1_FPI_ml-prove.csv"

In [13]:
indices = CSV.read(indice_file, DataFrame, header=false)[!, 1]
sample_name = splitext(basename(indice_file))[1]
parts = split(sample_name, '_')
sample_number = parts[3]

"1"

In [14]:
sample_trees = [graphviz_tree_list[i] for i in indices if i < length(graphviz_tree_list)]
n_trees = length(sample_trees)
tree_pairs = generate_pairs(n_trees)

43071-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 6)
 (1, 7)
 (1, 8)
 (1, 9)
 (1, 10)
 (1, 11)
 ⋮
 (290, 292)
 (290, 293)
 (290, 294)
 (291, 292)
 (291, 293)
 (291, 294)
 (292, 293)
 (292, 294)
 (293, 294)

In [15]:
output_hdf5_file = joinpath(output_dir, f"dist_edit_tree_{base_name}_sample_{sample_number}.h5")

"..\\Data\\Distancias\\Edit Tree Teste Julia\\dist_edit_tree_graphviz_tree_no_tag_FPI_ml-prove_sample_1.h5"

In [26]:

results = Tuple{Tuple{Int, Int}, Float64}[]

for (i, j) in tree_pairs
    tree1 = sample_trees[i]
    tree2 = sample_trees[j]
    distance = simple_distance(Node(tree1), Node(tree2))
    push!(results, ((i-1, j-1), distance))
end



In [27]:
results

43071-element Vector{Tuple{Tuple{Int64, Int64}, Float64}}:
 ((0, 1), 0.0)
 ((0, 2), 0.0)
 ((0, 3), 0.0)
 ((0, 4), 0.0)
 ((0, 5), 0.0)
 ((0, 6), 0.0)
 ((0, 7), 0.0)
 ((0, 8), 0.0)
 ((0, 9), 0.0)
 ((0, 10), 0.0)
 ⋮
 ((289, 291), 0.0)
 ((289, 292), 0.0)
 ((289, 293), 0.0)
 ((290, 291), 0.0)
 ((290, 292), 0.0)
 ((290, 293), 0.0)
 ((291, 292), 0.0)
 ((291, 293), 0.0)
 ((292, 293), 0.0)

In [16]:
sample_trees[tree_pairs[1][2]]

"digraph Program {\n\tnode [style=filled]\n\t0 [label=<Start> fillcolor=\"#E68EFB\"]\n\t1 [label=<preprocessing> fillcolor=\"#E68EFB\"]\n\t2 [label=NoPreprocessing fillcolor=\"#8EE7FB\"]\n\t3 [label=<classification> fillcolor=\"#E68EFB\"]\n\t4 [label=KNearestNeighbors fillcolor=\"#8EE7FB\"]\n\t5" ⋯ 33 bytes ⋯ " [label=distance fillcolor=\"#8EE7FB\"]\n\t7 [label=ball_tree fillcolor=\"#8EE7FB\"]\n\t8 [label=60 fillcolor=\"#8EE7FB\"]\n\t9 [label=minkowski fillcolor=\"#8EE7FB\"]\n\t10 [label=3 fillcolor=\"#8EE7FB\"]\n\t0 -> 1\n\t1 -> 2\n\t0 -> 3\n\t3 -> 4\n\t3 -> 5\n\t3 -> 6\n\t3 -> 7\n\t3 -> 8\n\t3 -> 9\n\t3 -> 10\n}\n"

In [18]:
tree1 = sample_trees[tree_pairs[1][1]]
tree2 = sample_trees[tree_pairs[1][2]]
distance = simple_distance(Node(tree1), Node(tree2))

0.0